In [1]:
import datasets
import numpy as np
# import hyped components
from hyped.data.pipe import DataPipe
from hyped.data.processors.api.openai_chat import OpenAIChatCompletion, OpenAIChatCompletionConfig
from hyped.data.processors.templates.jinja2 import Jinja2, Jinja2Config
from hyped.data.processors.parsers.json import JsonParser, JsonParserConfig
from hyped.common.feature_key import FeatureCollection, Const, FeatureDict
# import matplotlib
import matplotlib.pyplot as plt
# load openai key from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
pipe = DataPipe(
    [
        Jinja2(
            Jinja2Config(
                template="{{ role }}",
                features=FeatureDict({
                    "role": Const("AI Assistant"),
                }),
                output="user_prompt",
            )
        ),
        # OpenAIChatCompletion(
        #     OpenAIChatCompletionConfig(
        #         messages=FeatureCollection(
        #             [
        #                 {
        #                     "role": Const("system"),
        #                     "content": Const(
        #                         "Given text excerpts from IMDb movie reviews, your task is to predict "
        #                         "whether each review is positive or negative. Your output should be a "
        #                         "json string of the following format: {\"label\": <LABEL>} where "
        #                         "<LABEL> is either \"positive\" or \"negative\"."
        #                     )
        #                 },
        #                 {
        #                     "role": Const("user"),
        #                     "content": "user_prompt"
        #                 }
        #             ]
        #         ),
        #         model="gpt-3.5-turbo-0125",
        #         response_format={"type": "json_object"},
        #     )
        # ),
        # JsonParser(
        #     JsonParserConfig(
        #         json_str=("completion", "message"),
        #         scheme=datasets.Features(
        #             {
        #                 "label": datasets.ClassLabel(
        #                     names=["negative", "positive"]
        #                 )
        #             }
        #         ),
        #         output_format={"pred": ("parsed", "label")}
        #     )
        # )
    ]
)

In [4]:
ds = datasets.load_dataset("stanfordnlp/imdb", split="test")
ds = ds.select(np.random.choice(len(ds), 10))
ds

Dataset({
    features: ['text', 'label'],
    num_rows: 10
})

In [5]:
# apply pipeline to dataset
out_ds = pipe.apply(ds)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [6]:
print(out_ds[0]["user_prompt"])

AI Assistant


In [5]:
preds = np.asarray(out_ds["pred"])
trues = np.asarray(out_ds["label"])
# compute accuracy
(preds == trues).astype(float).mean()

1.0